In [2]:
import requests
import json
import csv
import pandas as pd
from config import omdb_key, postgres_connection_string
from pprint import pprint

In [3]:
#import csv
musical_csv = "../Resources/Musical_List.csv"

In [4]:
#convert csv to dataframe
musical_movie_sales = pd.read_csv(musical_csv)
musical_movie_sales1 = musical_movie_sales.copy()
musical_movie_sales1.head()

,Rank,Released,Movie,Worldwide Box Office,Domestic Box Office,International Box Office
0,1,2013,Frozen,"$1,267,856,756","$400,953,009","$866,903,747"
1,2,2017,Beauty and the Beast,"$1,255,080,655","$504,014,165","$751,066,490"
2,3,2017,Coco,"$798,237,597","$210,460,015","$587,777,582"
3,4,2016,Moana,"$636,155,760","$248,757,044","$387,398,716"
4,5,2008,Mamma Mia!,"$603,604,074","$144,130,063","$459,474,011"


In [5]:
#set up omdb api connection
url = "http://www.omdbapi.com/?t="
api_key_param = "&apikey=" + omdb_key


In [6]:
#pull data from api
response = requests.get(url + "Frozen" + api_key_param)
data = response.json()
pprint(data)

{'Actors': 'Kristen Bell, Idina Menzel, Jonathan Groff, Josh Gad',
 'Awards': 'Won 2 Oscars. Another 79 wins & 60 nominations.',
 'BoxOffice': '$400,953,009',
 'Country': 'USA, Norway',
 'DVD': '15 Nov 2015',
 'Director': 'Chris Buck, Jennifer Lee',
 'Genre': 'Animation, Adventure, Comedy, Family, Fantasy, Musical',
 'Language': 'English, Norwegian',
 'Metascore': '75',
 'Plot': 'When the newly crowned Queen Elsa accidentally uses her power to '
         'turn things into ice to curse her home in infinite winter, her '
         'sister Anna teams up with a mountain man, his playful reindeer, and '
         'a snowman to change the weather condition.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTQ1MjQwMTE5OF5BMl5BanBnXkFtZTgwNjk3MTcyMDE@._V1_SX300.jpg',
 'Production': 'Walt Disney Animation',
 'Rated': 'PG',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.4/10'},
             {'Source': 'Rotten Tomatoes', 'Value': '90%'},
             {'Source': 'Metacritic', 'V

In [45]:
#Add data to dataframe using loop
movie_data_df = []
movie_info = {}

for index,row in musical_movie_sales1.iterrows():
    try:
        movie_name = row['Movie']
        response = requests.get(url + movie_name + api_key_param)
        data = response.json()
        if data['Response']== 'False':
            continue
             
    except:
        print('error1')
        print(data['Response'])
        break
    try:
        movie_info['Rating'] = data.get('Metascore')
        print(data.get('Metascore'))
    except:
        print(f'No Movie score for {movie_name} found')
        print(data.get('Metascore'))
        break

    try:
        movie_info['Director'] = data.get('Director')
        print(data.get('Director'))
    except:
        print(f'No directors for {movie_name} found')
        print(data.get('Director'))
        break
    try:
        movie_info['Movie']= movie_name
        print(movie_name)
    except:
        print('error2')

        break
    try:
        actors = data.get("Actors")
        movie_info['Actors'] = actors    
    except:
        print('error3')
        print(actors)
        print(data)
        break
    movie_data_df.append(movie_info.copy())

75
Chris Buck, Jennifer Lee
Frozen
95
Gary Trousdale, Kirk Wise
Beauty and the Beast
81
Lee Unkrich, Adrian Molina(co-director)
Coco
81
Ron Clements, John Musker, Don Hall(co-director), Chris Williams(co-director)
Moana
51
Phyllida Lloyd
Mamma Mia!
71
Nathan Greno, Byron Howard
Tangled
94
Damien Chazelle
La La Land
95
Gary Trousdale, Kirk Wise
Beauty and the Beast
63
Tom Hooper
Les Misérables
48
Michael Gracey
The Greatest Showman
70
Randal Kleiser
Grease
66
Rob Marshall
Mary Poppins Returns
81
Rob Marshall
Chicago
63
Robert Wise
The Sound of Music
67
Kenneth Branagh
Cinderella
69
Rob Marshall
Into the Woods
81
Adam Shankman
Hairspray
95
William Cottrell, David Hand, Wilfred Jackson, Larry Morey, Perce Pearce, Ben Sharpsteen
Snow White and the Seven Dwarfs
N/A
John Huston
Moulin Rouge
76
Bill Condon
Dreamgirls
40
Joel Schumacher
The Phantom of the Opera
83
Tim Burton
Sweeney Todd: The Demon Barber of Fleet Street
45
Alan Parker
Evita
77
Jon Favreau
The Jungle Book
39
John Huston
Annie


In [46]:
movie_data_df[0:10]

[{'Rating': '75',
  'Director': 'Chris Buck, Jennifer Lee',
  'Movie': 'Frozen',
  'Actors': 'Kristen Bell, Idina Menzel, Jonathan Groff, Josh Gad'},
 {'Rating': '95',
  'Director': 'Gary Trousdale, Kirk Wise',
  'Movie': 'Beauty and the Beast',
  'Actors': 'Robby Benson, Jesse Corti, Rex Everhart, Angela Lansbury'},
 {'Rating': '81',
  'Director': 'Lee Unkrich, Adrian Molina(co-director)',
  'Movie': 'Coco',
  'Actors': 'Anthony Gonzalez, Gael García Bernal, Benjamin Bratt, Alanna Ubach'},
 {'Rating': '81',
  'Director': 'Ron Clements, John Musker, Don Hall(co-director), Chris Williams(co-director)',
  'Movie': 'Moana',
  'Actors': "Auli'i Cravalho, Dwayne Johnson, Rachel House, Temuera Morrison"},
 {'Rating': '51',
  'Director': 'Phyllida Lloyd',
  'Movie': 'Mamma Mia!',
  'Actors': 'Amanda Seyfried, Stellan Skarsgård, Pierce Brosnan, Nancy Baldwin'},
 {'Rating': '71',
  'Director': 'Nathan Greno, Byron Howard',
  'Movie': 'Tangled',
  'Actors': 'Mandy Moore, Zachary Levi, Donna Murp

In [49]:
new_movie_data_df = pd.DataFrame.from_dict(movie_data_df, orient = 'columns')
new_movie_data_df = new_movie_data_df[['Movie', 'Director','Actors', 'Rating']]
new_movie_data_df.head()

,Movie,Director,Actors,Rating
0,Frozen,"Chris Buck, Jennifer Lee","Kristen Bell, Idina Menzel, Jonathan Groff, Jo...",75
1,Beauty and the Beast,"Gary Trousdale, Kirk Wise","Robby Benson, Jesse Corti, Rex Everhart, Angel...",95
2,Coco,"Lee Unkrich, Adrian Molina(co-director)","Anthony Gonzalez, Gael García Bernal, Benjamin...",81
3,Moana,"Ron Clements, John Musker, Don Hall(co-directo...","Auli'i Cravalho, Dwayne Johnson, Rachel House,...",81
4,Mamma Mia!,Phyllida Lloyd,"Amanda Seyfried, Stellan Skarsgård, Pierce Bro...",51


In [7]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
engine = create_engine(f'postgresql://{postgres_connection_string}')

ModuleNotFoundError: No module named 'sqlalchemy_utils'

In [4]:
csv_df = 'new_movie_data_df.csv'
new_movie_data_df = pd.read_csv(csv_df)
new_movie_data_df.head()

,Unnamed: 0,Movie,Director,Actors,Rating
0,0,Frozen,"Chris Buck, Jennifer Lee","Kristen Bell, Idina Menzel, Jonathan Groff, Jo...",75.0
1,1,Beauty and the Beast,"Gary Trousdale, Kirk Wise","Robby Benson, Jesse Corti, Rex Everhart, Angel...",95.0
2,2,Coco,"Lee Unkrich, Adrian Molina(co-director)","Anthony Gonzalez, Gael García Bernal, Benjamin...",81.0
3,3,Moana,"Ron Clements, John Musker, Don Hall(co-directo...","Auli'i Cravalho, Dwayne Johnson, Rachel House,...",81.0
4,4,Mamma Mia!,Phyllida Lloyd,"Amanda Seyfried, Stellan Skarsgård, Pierce Bro...",51.0


In [55]:
new_movie_data_df.to_csv('new_movie_data_df.csv')

In [6]:
# Create database if it does not exist.
if not database_exists(engine.url):
    create_database(engine.url)
else:
    # Connect the database if exists.
    engine.connect()
new_movie_data_df.to_sql(name = "Movie_Info", con=engine, if_exists='replace', index = False)

NameError: name 'database_exists' is not defined